In [1]:
pip install scikit-multilearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 1.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import skmultilearn
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score,hamming_loss,classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.adapt import MLkNN

In [ ]:
df=pd.read_csv('/kaggle/input/paper-prophet-challenge/trainingDATA.csv')
df.head()

In [ ]:
def labeling(s):
  if(type(s)==list):
    return s
  if(type(s)==float):
    return str(s).split(' ')
  return s.split(' ')

In [ ]:
df['categories']=df['categories'].apply(labeling)

In [ ]:
multilabel=MultiLabelBinarizer()
y=multilabel.fit_transform(df['categories'])
y

In [ ]:
df2=pd.DataFrame(y,columns=multilabel.classes_)

In [ ]:
df2.dtypes

In [ ]:
tfidf = TfidfVectorizer()

In [ ]:
tfidf=TfidfVectorizer(analyzer='word',max_features=10000,ngram_range=(1,2),stop_words='english')

In [ ]:
x_train = tfidf.fit_transform(df[' title'].values.astype('U'))

In [ ]:
print(x_train)

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPool1D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D
import tensorflow as tf
filter_length = 300

model = Sequential()
model.add(Embedding(max_words, 20, input_length=maxlen))
model.add(Conv1D(filter_length, 3, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPool1D())
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC()])

callbacks = [
    ReduceLROnPlateau(),
    ModelCheckpoint(filepath='model-conv1d.h5', save_best_only=True)
]

history = model.fit(X_train, y_train,
                    epochs=25,
                    batch_size=32,
                    validation_split=0.3,
                    callbacks=callbacks)

In [ ]:
df2=pd.read_csv('/kaggle/input/paper-prophet-challenge/testingDATA .csv')
df2.head()

In [ ]:
x_test=tfidf.fit_transform(df2[' title'].values.astype('U'))

In [ ]:
print(x_test)

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
print(y_pred)

In [ ]:
y_pred.toarray()

In [ ]:
y_pred=y_pred.toarray()

In [ ]:
sub=pd.DataFrame(y_pred,columns=multilabel.classes_)
sub

In [ ]:
sub['categories']=(sub.stack().loc[lambda x: x>0].reset_index().groupby('level_0').agg({'level_1':list}))


In [ ]:
sub

In [ ]:
subf=pd.DataFrame(sub['categories'])

In [ ]:
subf.insert(0,"ID",df2['ID'],True)

In [ ]:
subf

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
subf.to_csv('/content/gdrive/My Drive/Submission2.csv', index=False)
print(pd.read_csv('/content/gdrive/My Drive/Submission.csv'))